In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
import time
import pandas as pd
from scipy.stats import pearsonr

In [53]:
from model.util import Normalizer
from model.database_util import get_hist_file, get_job_table_sample, collator
from model.model import QueryFormer
from model.database_util import Encoding
from model.dataset import PlanTreeDataset
from model.trainer import eval_workload, train

In [3]:
data_path = './data/imdb/'

In [48]:
class Args:
    bs = 1024
    lr = 0.001
    # SQ: changed the epochs to 1 to run faster
    # epochs = 200
    epochs = 1
    clip_size = 50
    embed_size = 64 
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    # SQ: changed to cpu
    # device = 'cuda:0'
    device = 'cpu'
    newpath = './results/full/cost/'
    to_predict = 'cost'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)
print(args.newpath)

./results/full/cost/


In [5]:
# What is in this histogram file?
hist_file = get_hist_file(data_path + 'histogram_string.csv')


/home/db2inst1/sq-wlm/QueryFormer/model/database_util.py:108: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  hist_file['bins'][rid] = \
/home/db2inst1/sq-wlm/QueryFormer/model/database_util.py:108: FutureWarning: ChainedAssignmentError: behav

In [6]:
hist_file.shape

(9, 5)

In [7]:
hist_file.head(10)

,table,column,freq,bins,table_column
0,title,production_year,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1880, 1913, 1923, 1942, 1955, 1960, 1964, 196...",t.production_year
1,title,kind_id,"[0.0, 0.26216118191156074, 0.03593387047716835...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, ...",t.kind_id
2,movie_companies,company_id,"[0.0, 0.0004959511376981121, 0.000558807386989...","[1, 6, 19, 27, 68, 133, 160, 189, 292, 402, 47...",mc.company_id
3,movie_companies,company_type_id,"[0.0, 0.4883796425472418, 0.5116203574527581]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",mc.company_type_id
4,cast_info,role_id,"[0.0, 0.3495907485479872, 0.20560375449487386,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",ci.role_id
5,movie_keyword,keyword_id,"[0.0, 0.0031748950967179193, 1.989421142551088...","[1, 77, 132, 230, 331, 347, 384, 495, 643, 784...",mk.keyword_id
6,cast_info,person_id,"[0.0, 0.0, 5.518102507748589e-08, 2.7590512538...","[2, 77446, 145798, 212750, 281691, 347240, 419...",ci.person_id
7,movie_info_idx,info_type_id,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 9...",mi_idx.info_type_id
8,movie_info,info_type_id,"[0.0, 0.05406815807174563, 0.08688004942665738...","[1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, ...",mi.info_type_id


### Questions on hist_file:
1. What does this file contain?
2. Why are there two columns with the same value: i.e., column and table_column?
3. What are frequencies?
4. What are bins?

In [8]:
cost_norm = Normalizer(-3.61192, 12.290855)
card_norm = Normalizer(1,100)

In [9]:
encoding_ckpt = torch.load('checkpoints/encoding.pt')
# SQ: what's in the encoding.pt file?

In [10]:
encoding_ckpt.keys()
# Whose encoding?


dict_keys(['encoding'])

In [11]:
encoding_ckpt

{'encoding': <model.database_util.Encoding at 0x7fd67c84a8d0>}

In [12]:
for key, value in encoding_ckpt.items():
    print(key)
    print(value)

encoding


In [13]:

# Iterate over the key-value pairs
for key, value in encoding_ckpt.items():
    
    print(f"col-min-max: {value.column_min_max_vals}")
    print("---------")
    print(f"col-index: {value.col2idx}")
    print("---------")
    print(f"op-index: {value.op2idx}")
    print("---------")
    print(f"type-index: {value.type2idx}")

col-min-max: {'t.id': [1.0, 2528312.0], 't.kind_id': [1.0, 7.0], 't.production_year': [1880.0, 2019.0], 'mc.id': [1.0, 2609129.0], 'mc.company_id': [1.0, 234997.0], 'mc.movie_id': [2.0, 2525745.0], 'mc.company_type_id': [1.0, 2.0], 'ci.id': [1.0, 36244344.0], 'ci.movie_id': [1.0, 2525975.0], 'ci.person_id': [1.0, 4061926.0], 'ci.role_id': [1.0, 11.0], 'mi.id': [1.0, 14835720.0], 'mi.movie_id': [1.0, 2526430.0], 'mi.info_type_id': [1.0, 110.0], 'mi_idx.id': [1.0, 1380035.0], 'mi_idx.movie_id': [2.0, 2525793.0], 'mi_idx.info_type_id': [99.0, 113.0], 'mk.id': [1.0, 4523930.0], 'mk.movie_id': [2.0, 2525971.0], 'mk.keyword_id': [1.0, 134170.0]}
---------
col-index: {'t.id': 0, 't.kind_id': 1, 't.production_year': 2, 'mc.id': 3, 'mc.company_id': 4, 'mc.movie_id': 5, 'mc.company_type_id': 6, 'ci.id': 7, 'ci.movie_id': 8, 'ci.person_id': 9, 'ci.role_id': 10, 'mi.id': 11, 'mi.movie_id': 12, 'mi.info_type_id': 13, 'mi_idx.id': 14, 'mi_idx.movie_id': 15, 'mi_idx.info_type_id': 16, 'mk.id': 17, 'm

In [14]:
encoding = encoding_ckpt['encoding']
# SQ: what is in encoding? 
# SQ: how was this encoding created?
# SQ: Is this encoding specific to a workload? or a dataset?


In [15]:
type(encoding)

model.database_util.Encoding

In [16]:
# SQ: from looking into the definition of Encoding class, it appears that its a data structure for 
# storing the encoding information of a dataset? 

In [17]:
checkpoint = torch.load('checkpoints/cost_model.pt', map_location='cpu')
# SQ: what is the cost model?
# SQ: what is being loaded here?

In [18]:
type(checkpoint)

dict

In [19]:
# checkpoint.keys()
print(checkpoint.keys())



print(checkpoint["args"].bs)
print(checkpoint["args"].lr)
print(checkpoint["args"].epochs)
print(checkpoint["args"].clip_size)
print(checkpoint["args"].embed_size)
print(checkpoint["args"].pred_hid)
print(checkpoint["args"].ffn_dim)

# YA: They store the model and its hyperparameter in Args class


dict_keys(['model', 'args'])
1024
0.001
1
50
64
128
128


In [20]:
# SQ: Is this model already trained? 

In [21]:
from model.util import seed_everything
seed_everything()
# This function seed_everything() is used to set a fixed seed for random number generators in Python, NumPy, and PyTorch. 
# This is done to ensure that the results of your code are reproducible.

In [22]:
model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = True, use_hist = True, \
                 pred_hid = args.pred_hid
                )
# emb_size = length of each embedding vector. Will we have 1 embedding vector per node in the query plan tree?
# embeddings are used to convert categorical variables into continuous ones that can be processed by the model.
# ffn_dim = the number of nodes in each hidden layer

In [23]:
type(model)

model.model.QueryFormer

In [24]:
print(model)

QueryFormer(
  (rel_pos_encoder): Embedding(64, 12, padding_idx=0)
  (height_encoder): Embedding(64, 329, padding_idx=0)
  (input_dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0-7): 8 x EncoderLayer(
      (self_attention_norm): LayerNorm((329,), eps=1e-05, elementwise_affine=True)
      (self_attention): MultiHeadAttention(
        (linear_q): Linear(in_features=329, out_features=324, bias=True)
        (linear_k): Linear(in_features=329, out_features=324, bias=True)
        (linear_v): Linear(in_features=329, out_features=324, bias=True)
        (att_dropout): Dropout(p=0.1, inplace=False)
        (output_layer): Linear(in_features=324, out_features=329, bias=True)
      )
      (self_attention_dropout): Dropout(p=0.1, inplace=False)
      (ffn_norm): LayerNorm((329,), eps=1e-05, elementwise_affine=True)
      (ffn): FeedForwardNetwork(
        (layer1): Linear(in_features=329, out_features=128, bias=True)
        (gelu): GELU(approximate='none')
        (layer

In [25]:
_ = model.to(args.device)

In [26]:
to_predict = 'cost'

## SQ: Loading training set
The following code block is loading the training set from a set of files and concatenating them to produce a final dataframe training set
1. What's the structure of the training set?

In [27]:
imdb_path = './data/imdb/'
full_train_df = pd.DataFrame()
#for i in range(18):
for i in range(2):
    file = imdb_path + 'plan_and_cost/train_plan_part{}.csv'.format(i)
    df = pd.read_csv(file)
    # print(df.shape)
    ## SQ: the following line of coding is giving this error: AttributeError: 'DataFrame' object has no attribute 'append'
    # full_train_df = full_train_df.append(df)
    # SQ: changing the code
    full_train_df = pd.concat([full_train_df, df])

print(full_train_df['json'])



0       {"Plan": {"Node Type": "Gather", "Parallel Awa...
1       {"Plan": {"Node Type": "Seq Scan", "Parallel A...
2       {"Plan": {"Node Type": "Seq Scan", "Parallel A...
3       {"Plan": {"Node Type": "Gather", "Parallel Awa...
4       {"Plan": {"Node Type": "Bitmap Heap Scan", "Pa...
                              ...                        
4995    {"Plan": {"Node Type": "Hash Join", "Parallel ...
4996    {"Plan": {"Node Type": "Gather", "Parallel Awa...
4997    {"Plan": {"Node Type": "Gather", "Parallel Awa...
4998    {"Plan": {"Node Type": "Gather", "Parallel Awa...
4999    {"Plan": {"Node Type": "Gather", "Parallel Awa...
Name: json, Length: 10000, dtype: object


In [28]:
val_df = pd.DataFrame()
for i in range(18,20):
    file = imdb_path + 'plan_and_cost/train_plan_part{}.csv'.format(i)
    df = pd.read_csv(file)
    ## SQ: the following line of coding is giving this error: AttributeError: 'DataFrame' object has no attribute 'append'
    # val_df = val_df.append(df)
    # SQ: changed the following to the following
    val_df = pd.concat([val_df, df])
table_sample = get_job_table_sample(imdb_path+'train')

Loaded queries with len  100000
Loaded bitmaps


In [29]:
# YA: Reading the first query plan
specific_value = full_train_df.iloc[0][1]
print(specific_value)

{"Plan": {"Node Type": "Gather", "Parallel Aware": false, "Startup Cost": 23540.58, "Total Cost": 154548.95, "Plan Rows": 567655, "Plan Width": 119, "Actual Startup Time": 386.847, "Actual Total Time": 646.972, "Actual Rows": 283812, "Actual Loops": 1, "Workers Planned": 2, "Workers Launched": 2, "Single Copy": false, "Plans": [{"Node Type": "Hash Join", "Parent Relationship": "Outer", "Parallel Aware": true, "Join Type": "Inner", "Startup Cost": 22540.58, "Total Cost": 96783.45, "Plan Rows": 236523, "Plan Width": 119, "Actual Startup Time": 369.985, "Actual Total Time": 518.487, "Actual Rows": 94604, "Actual Loops": 3, "Inner Unique": false, "Hash Cond": "(t.id = mi_idx.movie_id)", "Workers": [], "Plans": [{"Node Type": "Seq Scan", "Parent Relationship": "Outer", "Parallel Aware": true, "Relation Name": "title", "Alias": "t", "Startup Cost": 0.0, "Total Cost": 49166.46, "Plan Rows": 649574, "Plan Width": 94, "Actual Startup Time": 0.366, "Actual Total Time": 147.047, "Actual Rows": 51

/tmp/ipykernel_322315/1381683576.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  specific_value = full_train_df.iloc[0][1]


In [30]:
full_train_df.shape

(10000, 2)

In [31]:
full_train_df.head(5)

,id,json
0,0,"{""Plan"": {""Node Type"": ""Gather"", ""Parallel Awa..."
1,1,"{""Plan"": {""Node Type"": ""Seq Scan"", ""Parallel A..."
2,2,"{""Plan"": {""Node Type"": ""Seq Scan"", ""Parallel A..."
3,3,"{""Plan"": {""Node Type"": ""Gather"", ""Parallel Awa..."
4,4,"{""Plan"": {""Node Type"": ""Bitmap Heap Scan"", ""Pa..."


In [32]:
full_train_df.dtypes

id       int64
json    object
dtype: object

In [33]:
temp_df = full_train_df.head(10)

In [34]:
temp_df.head()

,id,json
0,0,"{""Plan"": {""Node Type"": ""Gather"", ""Parallel Awa..."
1,1,"{""Plan"": {""Node Type"": ""Seq Scan"", ""Parallel A..."
2,2,"{""Plan"": {""Node Type"": ""Seq Scan"", ""Parallel A..."
3,3,"{""Plan"": {""Node Type"": ""Gather"", ""Parallel Awa..."
4,4,"{""Plan"": {""Node Type"": ""Bitmap Heap Scan"", ""Pa..."


In [35]:
import json

In [36]:
temp_length = len(temp_df)
#nodes = [json.loads(plan) for plan in temp_df['json']]
nodes = [json.loads(plan)['Plan'] for plan in temp_df['json']]
print(nodes)

[{'Node Type': 'Gather', 'Parallel Aware': False, 'Startup Cost': 23540.58, 'Total Cost': 154548.95, 'Plan Rows': 567655, 'Plan Width': 119, 'Actual Startup Time': 386.847, 'Actual Total Time': 646.972, 'Actual Rows': 283812, 'Actual Loops': 1, 'Workers Planned': 2, 'Workers Launched': 2, 'Single Copy': False, 'Plans': [{'Node Type': 'Hash Join', 'Parent Relationship': 'Outer', 'Parallel Aware': True, 'Join Type': 'Inner', 'Startup Cost': 22540.58, 'Total Cost': 96783.45, 'Plan Rows': 236523, 'Plan Width': 119, 'Actual Startup Time': 369.985, 'Actual Total Time': 518.487, 'Actual Rows': 94604, 'Actual Loops': 3, 'Inner Unique': False, 'Hash Cond': '(t.id = mi_idx.movie_id)', 'Workers': [], 'Plans': [{'Node Type': 'Seq Scan', 'Parent Relationship': 'Outer', 'Parallel Aware': True, 'Relation Name': 'title', 'Alias': 't', 'Startup Cost': 0.0, 'Total Cost': 49166.46, 'Plan Rows': 649574, 'Plan Width': 94, 'Actual Startup Time': 0.366, 'Actual Total Time': 147.047, 'Actual Rows': 514421, 'A

In [37]:
type(nodes)

list

In [38]:
len(nodes)

10

In [39]:
cards = [node['Actual Rows'] for node in nodes]


In [40]:
cards

[283812,
 1107925,
 3624977,
 134807,
 54826,
 1172762,
 4007119,
 2609129,
 920110,
 1603946]

In [41]:
costs = [json.loads(plan)['Execution Time'] for plan in temp_df['json']]

In [42]:
costs

[654.241,
 349.797,
 1699.24,
 345.056,
 90.666,
 1969.649,
 451.819,
 2872.724,
 139.463,
 2214.235]

In [43]:
train_ds = PlanTreeDataset(full_train_df, None, encoding, hist_file, card_norm, cost_norm, to_predict, table_sample)
val_ds = PlanTreeDataset(val_df, None, encoding, hist_file, card_norm, cost_norm, to_predict, table_sample)

/home/db2inst1/sq-wlm/QueryFormer/model/dataset.py:109: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  'features' : torch.FloatTensor(features),


In [54]:
crit = nn.MSELoss()
model, best_path = train(model, train_ds, val_ds, crit, cost_norm, args)

Epoch: 0  Avg Loss: 9.878756431862712e-06, Time: 30.722636699676514
Median: 2.884076492072395
Mean: 7.973215881877905


UnboundLocalError: cannot access local variable 'best_model_path' where it is not associated with a value

In [ ]:
nodes = [json.loads(plan)['Plan'] for plan in train_ds['json']]
idxs = list(train_ds['id'])
train_ds = train_ds.traversePlan(idxs[0], nodes[0], train_ds.encoding)

In [15]:
methods = {
    'get_sample' : get_job_table_sample,
    'encoding': encoding,
    'cost_norm': cost_norm,
    'hist_file': hist_file,
    'model': model,
    'device': args.device,
    'bs': 512,
}

In [16]:
_ = eval_workload('job-light', methods)

Loaded queries with len  70
Loaded bitmaps
Median: 1.6015447359157347
Mean: 15.04861380976482
Corr:  0.8955015382416885


In [17]:
_ = eval_workload('synthetic', methods)

Loaded queries with len  5000
Loaded bitmaps
Median: 1.0554397104507522
Mean: 1.7017223965744472
Corr:  0.9835725288032631
